In [28]:
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline, FeatureUnion, make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler

from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score

from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score

In [39]:
train = pd.read_json('../data/json/train.json', orient='records', lines=True)
test = pd.read_json('../data/json/test.json', orient='records', lines=True)
valid = pd.read_json('../data/json/valid.json', orient='records', lines=True)

In [3]:
n_cats = len(train['category_id'].unique())
n_cats

54

In [49]:
train.head()

,cat_0,cat_1,cat_10,cat_11,cat_12,cat_13,cat_14,cat_15,cat_16,cat_17,...,cat_6,cat_7,cat_8,cat_9,category_id,item_id,lemmatized,price,text,tokens
0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.000000,42,92389,"[ботинок, горнолыжный, женский, продавать, бот...",3500,Ботинки горнолыжные (женские) Продам ботинки д...,"[ботинки, горнолыжные, женские, продам, ботинк..."
1,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.000000,33,81327,"[продавать, пальто, продавать, красный, пальто...",3000,Продам пальто Продам красное пальто.Очень крас...,"[продам, пальто, продам, красное, пальто, очен..."
2,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.076923,9,224694,"[чехол, для, meizu, m2, mini, продавать, чехол...",250,Чехол для Meizu m2 mini Продам чехол для Meizu...,"[чехол, для, meizu, m2, mini, продам, чехол, д..."
3,0.012048,0.0,0.012048,0.012048,0.0,0.0,0.0,0.0,0.0,0.0,...,0.012048,0.0,0.0,0.011765,51,319263,"[ирвин, ялом, 3, книга, комплект, книга, компл...",900,Ирвин Ялом - 3 книги комплектом Книги комплект...,"[ирвин, ялом, 3, книги, комплектом, книги, ком..."
4,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.000000,47,251147,"[аром, палочка, для, йога, от, 200, до, 500, р...",200,Арома палочки для йоги От 200 до 500 рублей,"[арома, палочки, для, йоги, от, 200, до, 500, ..."


In [55]:
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
SEED = 1984

In [56]:
logreg = LogisticRegression(solver='saga', multi_class='multinomial', n_jobs=4, random_state=SEED, penalty='l1')

In [64]:
sgd = SGDClassifier( penalty='elasticnet', random_state=SEED, n_jobs=4, early_stopping=True)

In [ ]:
tfidf_pipe = make_pipeline([
    TfidfVectorizer(tokenizer=lambda x:x, lowercase=False),
    MinMaxScaler()
])


In [60]:

column_trans = ColumnTransformer(
    [
        ('tfidf', TfidfVectorizer(tokenizer=lambda x:x, lowercase=False), 'lemmatized'), 
        ('scaler', MinMaxScaler(), [x for x in train.columns if 'cat_' in x] + ['price'])
    ])

In [61]:
pipe = make_pipeline(column_trans, logreg)

In [63]:
cross_val_score(pipe, train, train['category_id'], cv=5)

/usr/local/lib/python3.6/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/usr/local/lib/python3.6/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/usr/local/lib/python3.6/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/usr/local/lib/python3.6/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/usr/local/lib/python3.6/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all conve

array([0.87204983, 0.87492021, 0.8716312 , 0.87339897, 0.87133771])

In [ ]:
pipe_sgd = make_pipeline(column_trans, logreg)
cross_val_score(pipe_sgd, train, train['category_id'], cv=5,)

/usr/local/lib/python3.6/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/usr/local/lib/python3.6/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/usr/local/lib/python3.6/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/usr/local/lib/python3.6/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/usr/local/lib/python3.6/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all conve